File này: Huấn luyện model dựa trên dataset mới

In [20]:
import torch
from torch.utils.data import Dataset
import os
import numpy as np
import open3d as o3d

class PointCloudDataset(Dataset):
    def __init__(self, data_dir, num_points=2048):
        self.data_dir = data_dir
        self.samples = [os.path.join(data_dir, "pointclouds", f)
                        for f in os.listdir(os.path.join(data_dir, "pointclouds"))
                        if f.endswith(".ply")]
        self.num_points = num_points

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        try:
            # Load point cloud từ file .ply
            pcd = o3d.io.read_point_cloud(self.samples[idx])
            points = np.asarray(pcd.points)  # Shape: [N, 3] (x, y, z)

            # Kiểm tra điểm hợp lệ
            if points.shape[0] == 0:
                raise ValueError(f"Empty point cloud in {self.samples[idx]}")

            # Tính toán normal vectors nếu không có
            if not pcd.has_normals():
                pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
            normals = np.asarray(pcd.normals)  # Shape: [N, 3] (nx, ny, nz)

            # Kết hợp points và normals thành [N, 6]
            points = np.concatenate([points, normals], axis=1)  # Shape: [N, 6]

            # Validate points shape
            if points.shape[1] != 6:
                raise ValueError(f"Invalid points shape in {self.samples[idx]}: got {points.shape}, expected (N, 6)")

            # Subsample hoặc pad để có số điểm cố định
            num_points = points.shape[0]
            if num_points > self.num_points:
                # Randomly subsample points
                indices = np.random.choice(num_points, self.num_points, replace=False)
                points = points[indices]
            elif num_points < self.num_points:
                # Pad với các điểm lặp lại
                indices = np.random.choice(num_points, self.num_points - num_points, replace=True)
                points = np.concatenate([points, points[indices]], axis=0)
            # Now points.shape = [self.num_points, 6]

            # Xác định bưu kiện topmost dựa trên tọa độ z cao nhất
            z_coords = points[:, 2]  # Lấy cột z (tọa độ z)
            max_z = np.max(z_coords)
            topmost_mask = z_coords >= max_z - 0.01  # Ngưỡng nhỏ để lấy các điểm gần max z
            topmost_points = points[topmost_mask]

            # Kiểm tra nếu không có điểm topmost
            if len(topmost_points) == 0:
                raise ValueError(f"No topmost points found in {self.samples[idx]} with z >= {max_z - 0.01}")

            # Tính tâm và vector pháp tuyến của bưu kiện topmost
            center_gt = np.mean(topmost_points[:, :3], axis=0)  # Trung bình x, y, z
            normal_gt = np.mean(topmost_points[:, 3:], axis=0)  # Trung bình nx, ny, nz

            # Chuẩn hóa vector pháp tuyến, xử lý trường hợp zero vector
            norm = np.linalg.norm(normal_gt)
            if norm < 1e-8:  # Nếu norm quá nhỏ, trả về vector pháp tuyến mặc định
                normal_gt = np.array([0.0, 0.0, 1.0])  # Mặc định hướng lên (z-axis)
            else:
                normal_gt = normal_gt / norm

            return (torch.tensor(points, dtype=torch.float32),
                    torch.tensor(center_gt, dtype=torch.float32),
                    torch.tensor(normal_gt, dtype=torch.float32))
        except Exception as e:
            raise ValueError(f"Error processing {self.samples[idx]}: {str(e)}")

In [9]:
import os
print(os.listdir("dataset"))


['labels', 'pointclouds']


In [3]:
import torch
import torch.nn as nn

class SimplePointNet(nn.Module):
    def __init__(self):
        super(SimplePointNet, self).__init__()
        
        # MLP để xử lý từng điểm trong point cloud
        self.mlp = nn.Sequential(
            nn.Linear(6, 64),  # Input: 6 (x, y, z, nx, ny, nz)
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU()
        )
        
        # Fully connected layers để dự đoán tâm và pháp tuyến
        self.fc = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 6)  # Output: 3 (tâm: x, y, z) + 3 (pháp tuyến: nx, ny, nz)
        )

    def forward(self, x):
        # x: [B, N, 6]
        x = self.mlp(x)  # [B, N, 256]
        x = torch.max(x, dim=1)[0]  # Global Max Pool → [B, 256]
        out = self.fc(x)  # [B, 6]
        center_pred = out[:, :3]  # Tọa độ tâm
        normal_pred = out[:, 3:]  # Vector pháp tuyến
        # Chuẩn hóa vector pháp tuyến dự đoán
        normal_pred = normal_pred / (torch.norm(normal_pred, dim=-1, keepdim=True) + 1e-8)
        return center_pred, normal_pred

In [4]:
import torch.nn.functional as F

def compute_loss(center_pred, center_gt, normal_pred, normal_gt):
    # Tính loss cho tọa độ tâm bằng MSE
    loss_center = F.mse_loss(center_pred, center_gt)
    # Tính loss cho vector pháp tuyến bằng cosine similarity
    loss_normal = 1 - F.cosine_similarity(normal_pred, normal_gt, dim=-1).mean()
    
    return loss_center + loss_normal, loss_center, loss_normal

In [21]:
import torch
from torch.utils.data import DataLoader


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset = PointCloudDataset("./dataset")  # từ Module 5
loader = DataLoader(dataset, batch_size=8, shuffle=True)

model = SimplePointNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [12]:
print(device)

cuda


In [22]:
for epoch in range(10):
    model.train()
    total_loss, total_lc, total_ln = 0, 0, 0
    try:
        for points, center_gt, normal_gt in loader:
            points = points.to(device)      # [B, 2048, 6]
            center_gt = center_gt.to(device)  # [B, 3]
            normal_gt = normal_gt.to(device)  # [B, 3]

            # Dự đoán tâm và pháp tuyến
            center_pred, normal_pred = model(points)

            # Tính loss
            loss, lc, ln = compute_loss(center_pred, center_gt, normal_pred, normal_gt)

            # Backpropagation và cập nhật tham số
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            total_lc += lc.item()
            total_ln += ln.item()

        # In kết quả trung bình mỗi epoch
        avg_loss = total_loss / len(loader)
        avg_lc = total_lc / len(loader)
        avg_ln = total_ln / len(loader)
        print(f'Epoch {epoch+1}: total={avg_loss:.4f}, center={avg_lc:.4f}, normal={avg_ln:.4f}')
    except Exception as e:
        print(f"Error during training at epoch {epoch+1}: {e}")
        raise

Epoch 1: total=1.1540, center=0.4216, normal=0.7324
Epoch 2: total=1.1232, center=0.4002, normal=0.7230
Epoch 3: total=1.0367, center=0.3760, normal=0.6607
Epoch 4: total=1.0412, center=0.3632, normal=0.6780
Epoch 5: total=1.0153, center=0.3384, normal=0.6770
Epoch 6: total=0.9180, center=0.3188, normal=0.5992
Epoch 7: total=1.0221, center=0.3009, normal=0.7212
Epoch 8: total=0.9447, center=0.2915, normal=0.6532
Epoch 9: total=0.9845, center=0.2708, normal=0.7137
Epoch 10: total=1.0401, center=0.2574, normal=0.7827
